In [2]:
import tensorflow as tf
import pandas as pd




In [26]:

df = pd.DataFrame(pd.read_csv("data.csv", encoding='latin1'))
dates = df['InvoiceDate'].tolist()
for i in range(len(dates)):
    parts = dates[i].split(' ')[0].split('/')
    dates[i] = int(parts[0]) * (int(parts[2]) % 2) + 1
df['InvoiceDate'] = dates
print(df.tail())

       InvoiceNo StockCode                      Description  Quantity  \
541904    581587     22613      PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899     CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254    CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255  CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138    BAKING SET 9 PIECE RETROSPOT          3   

        InvoiceDate  UnitPrice  CustomerID Country  
541904           13       0.85     12680.0  France  
541905           13       2.10     12680.0  France  
541906           13       4.15     12680.0  France  
541907           13       4.15     12680.0  France  
541908           13       4.95     12680.0  France  


In [36]:

udf =df.groupby('StockCode')['Quantity'].sum().reset_index().sort_values('Quantity',ascending=False)
udf.head()


,StockCode,Quantity
1147,22197,56450
2997,84077,53847
3515,85099B,47363
3536,85123A,38830
3305,84879,36221


In [37]:
df['Weight'] = df['Quantity'] * df['InvoiceDate'] / 13
wdf =df.groupby('StockCode')['Weight'].sum().reset_index().sort_values('Weight',ascending=False)
wdf.head()

,StockCode,Weight
1147,22197,37439.153846
2997,84077,28529.384615
3515,85099B,26956.153846
1982,23084,26418.615385
3305,84879,21316.769231
